In [107]:
import bsscraper
import importlib
importlib.reload(bsscraper)

<module 'bsscraper' from '/Users/stevennewton/Documents/GitHub/MLB_Statcast/notebooka/bsscraper.py'>

In [ ]:
# Database for two years and two teams
bsscraper.make_database('NYY_NYM_19_20.db', [2019,2020], ['NYY,NYM'])

Every pitch thrown by the 2019 and 2020 New York Yankees and New York Mets is in a database. In the next step, I pick out interesting columns and decide to only include 2020 data. I also exclude empty columns which are deprecated stats. In 2020, Major League Baseball switched to a Hawk-Eye tracking system because it was slightly more accurate than Trackman. I'll still have more than enough observations for an analysis if I use 2020 by itself. (Lower down, I accidently deleted events because most of the values are null as they should be so i had to reload the database. It only takes a sec.)

In [692]:
import sqlite3
import pandas as pd
import time

t0  = time.time()
# Connect to, query, and load database into dataframe.
conn = sqlite3.connect('NYY_NYM_19_20_.db')
df = pd.read_sql_query('''
                        SELECT player_name, pitcher, game_year, game_date, game_type, home_team, away_team,
                        pitch_type, pitch_name, pitch_number, release_speed, release_spin_rate, vx0, vy0, vz0,
                        effective_speed, events, type, description, balls, strikes, zone, release_pos_x,
                        release_pos_z, pfx_x, pfx_z, plate_x, plate_z, release_extension, ax, ay, az
                        FROM statcast
                        WHERE pitch_type IS NOT NULL and game_year IS 2020;
                       '''
                       , conn)
conn.close()

print(f'\n That took {round(time.time()-t0, 5)} seconds!\n')
df2 = df.copy()
df2


 That took 0.93197 seconds!



,player_name,pitcher,game_year,game_date,game_type,home_team,away_team,pitch_type,pitch_name,pitch_number,...,release_pos_x,release_pos_z,pfx_x,pfx_z,plate_x,plate_z,release_extension,ax,ay,az
0,Aroldis Chapman,547973,2020,2020-10-09,D,TB,NYY,SL,Slider,3,...,0.76,6.07,-1.07,0.03,0.22,3.29,7.0,-11.171542,29.022734,-32.095575
1,Aroldis Chapman,547973,2020,2020-10-09,D,TB,NYY,SL,Slider,2,...,0.63,6.03,-0.93,0.31,-0.66,2.26,7.4,-9.423957,27.346872,-28.442330
2,Aroldis Chapman,547973,2020,2020-10-09,D,TB,NYY,FF,4-Seam Fastball,1,...,0.63,5.99,0.69,1.45,-0.31,2.36,7.3,10.198890,34.624279,-11.155418
3,Aroldis Chapman,547973,2020,2020-10-09,D,TB,NYY,FF,4-Seam Fastball,3,...,0.48,5.97,0.42,1.50,-0.93,2.65,7.2,6.877947,33.639729,-10.192959
4,Aroldis Chapman,547973,2020,2020-10-09,D,TB,NYY,FF,4-Seam Fastball,2,...,0.63,5.94,0.48,1.54,0.37,3.60,7.3,6.900246,34.172155,-10.546531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18611,Jacob deGrom,594798,2020,2020-07-24,R,NYM,ATL,FF,4-Seam Fastball,5,...,-1.08,5.53,-0.31,1.33,1.56,3.31,7.0,-6.125120,31.063964,-12.569235
18612,Jacob deGrom,594798,2020,2020-07-24,R,NYM,ATL,FF,4-Seam Fastball,4,...,-1.22,5.51,-0.74,1.39,-0.42,2.72,6.9,-11.223951,35.264626,-11.820192
18613,Jacob deGrom,594798,2020,2020-07-24,R,NYM,ATL,FF,4-Seam Fastball,3,...,-1.15,5.50,-0.61,1.28,1.56,3.34,7.0,-10.585942,32.953592,-13.606505
18614,Jacob deGrom,594798,2020,2020-07-24,R,NYM,ATL,FF,4-Seam Fastball,2,...,-1.23,5.48,-0.59,1.32,0.82,2.41,7.2,-9.820061,32.272418,-12.570117


In [312]:
#To verify this has every pitch, I'll estimate how many rows I expect for 2 teams in 2020.
est_pitches_per_game = 150
games = (2*60)+7   # shortened 2020 season plus postseason
f'I expected approximately {est_pitches_per_game*games} pitches'

'I expected approximately 19050 pitches'

In [341]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18616 entries, 0 to 18615
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   player_name        18616 non-null  object 
 1   pitcher            18616 non-null  int64  
 2   game_year          18616 non-null  int64  
 3   game_date          18616 non-null  object 
 4   game_type          18616 non-null  object 
 5   home_team          18616 non-null  object 
 6   away_team          18616 non-null  object 
 7   pitch_type         18616 non-null  object 
 8   pitch_name         18616 non-null  object 
 9   pitch_number       18616 non-null  int64  
 10  release_speed      18616 non-null  float64
 11  release_spin_rate  18615 non-null  float64
 12  vx0                18616 non-null  float64
 13  vy0                18616 non-null  float64
 14  vz0                18616 non-null  float64
 15  effective_speed    18616 non-null  float64
 16  events             465

In [339]:
df['pitch_type'].unique()

array(['SL', 'FF', 'FS', 'SI', 'KC', 'CH', 'CU', 'FC', 'KN', 'CS'],
      dtype=object)

In [340]:
df['pitch_name'].unique()

array(['Slider', '4-Seam Fastball', 'Split-Finger', 'Sinker',
       'Knuckle Curve', 'Changeup', 'Curveball', 'Cutter', 'Knuckleball'],
      dtype=object)

I know what all the codes are except CS. Apparently it's a single curveball Michael King threw August 30. I change the CS to CU. Then I'm going to make pitch types into a sparse array for easier grouping. This is usually as a pre-processing step for machine learning algorithms called one-hot encoding. It makes categorical variables numeric.

I'll do the same thing for descriptions and events after I combine them into one variable. Events are results of the plate appearance which mean the other pitches can be backfilled with descriptions (e.g. 'swinging-strike'.)

In [338]:
df[df['pitch_type'] =='CS']

,player_name,pitcher,game_year,game_date,game_type,home_team,away_team,pitch_type,pitch_name,pitch_number,...,release_pos_x,release_pos_z,pfx_x,pfx_z,plate_x,plate_z,release_extension,ax,ay,az
5489,Michael King,650633,2020,2020-08-30,R,NYY,NYM,CS,Curveball,1,...,-1.41,6.07,-0.03,1.5,0.22,6.37,7.0,-0.508284,11.843688,-26.880928


In [693]:
df2.at[5489, 'pitch_type'] = 'CU'

from sklearn.preprocessing import OneHotEncoder
import numpy as np

enc = OneHotEncoder(dtype=int, sparse=False)
x = np.array(df2['pitch_type']).reshape(-1, 1)

#Encoder outputs a 2d array. I reattach the header and put the array into a datframe.
arr = enc.fit_transform(x)
arr = np.concatenate((enc.categories_, arr))
arr = pd.DataFrame(arr[1:], columns=arr[0], dtype=int)
arr

,CH,CU,FC,FF,FS,KC,KN,SI,SL
0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,1
2,0,0,0,1,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
18611,0,0,0,1,0,0,0,0,0
18612,0,0,0,1,0,0,0,0,0
18613,0,0,0,1,0,0,0,0,0
18614,0,0,0,1,0,0,0,0,0


In [336]:
df.description.unique()

array(['called_strike', 'swinging_strike', 'hit_into_play', 'ball',
       'hit_into_play_score', 'foul', 'hit_into_play_no_out',
       'blocked_ball', 'swinging_strike_blocked', 'foul_tip',
       'hit_by_pitch', 'foul_bunt', 'missed_bunt', 'pitchout'],
      dtype=object)

In [337]:
df.events.unique()

array(['strikeout', None, 'field_out', 'home_run', 'field_error', 'walk',
       'single', 'hit_by_pitch', 'force_out', 'double',
       'grounded_into_double_play', 'double_play', 'sac_bunt', 'sac_fly',
       'fielders_choice', 'interf_def', 'strikeout_double_play',
       'fielders_choice_out', 'triple', 'other_out', 'pickoff_2b',
       'caught_stealing_2b', 'run', 'pickoff_caught_stealing_2b',
       'caught_stealing_3b'], dtype=object)

I don't need all those categories. I just want to know what the pitch accomplished. (Initially, I destroyed these records by making EVERYTHING in the row equal to the column values I wanted to put in. I was trying to figure out why empty records entered the data.) Replacing certain column values turned out to be a task and a half.

In [694]:
df2.loc[:,'events'].replace(['caught_stealing_2b', 'caught_stealing_3b', 'pickoff_caught_stealing_2b',
                             'run', 'pickoff_2b', 'other_out', 'strikeout_double_play', 'interf_def', 
                             'fielders_choice_out', 'fielders_choice', 'grounded_into_double_play',
                             'force_out', 'double_play', 'sac_bunt', 'sac_fly', 'hit_by_pitch'], 
                            [None, None, None, None, None, None, 'strikeout', 'field_error','field_out', 'field_out',
                             'field_out', 'field_out', 'field_out', 'field_out', 'field_out', 'walk'], inplace=True)

df2.loc[:,'description'].replace(['foul_bunt', 'swinging_strike_blocked', 'missed_bunt', 'foul_tip', 'foul',
                                  'blocked_ball', 'pitchout'], ['swinging_strike', 'swinging_strike',
                                        'swinging_strike', 'swinging_strike', 'swinging_strike','ball',
                                                                        'ball'], inplace=True)

In [572]:
def backwards_k(events, pitch):
    '''
    Add strikeouts looking to events.
    '''
    for ix, x in enumerate(events):
        if events[ix] == 'strikeout':
            if pitch[ix] == 'called_strike':
                events[ix] = events[ix] + '_looking' 

In [573]:
def backfill(col_to_fill, col_to_fill_from):
    '''
    Filling in None values with another column.
    '''
    for ix, x in enumerate(col_to_fill):
        if x == None:
            col_to_fill[ix] = col_to_fill_from[ix]

In [695]:
backwards_k(df2.events, df2.description)
backfill(df2.events, df2.description)

<ipython-input-572-2198a9a7a7ba>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events[ix] = events[ix] + '_looking'
<ipython-input-573-af22315491a5>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col_to_fill[ix] = col_to_fill_from[ix]


In [696]:
#Events and description columns are compressed into a manageable list.
df2.events.unique()

array(['strikeout_looking', 'swinging_strike', 'called_strike',
       'field_out', 'ball', 'home_run', 'field_error', 'strikeout',
       'walk', 'single', 'double', 'triple'], dtype=object)

In [697]:
enc = OneHotEncoder(dtype=int, sparse=False)
z = np.array(df2['events']).reshape(-1, 1)

#Encoder outputs a 2d array. I reattach the header and put the array into a datframe.
arr2 = enc.fit_transform(z)
arr2 = np.concatenate((enc.categories_, arr2))
arr2 = pd.DataFrame(arr2[1:], columns=arr2[0], dtype=int)
arr2

,ball,called_strike,double,field_error,field_out,home_run,single,strikeout,strikeout_looking,swinging_strike,triple,walk
0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
18611,1,0,0,0,0,0,0,0,0,0,0,0
18612,0,0,0,0,0,0,0,0,0,1,0,0
18613,1,0,0,0,0,0,0,0,0,0,0,0
18614,0,0,0,0,0,0,0,0,0,1,0,0


In [699]:
df2 = pd.concat([df2, arr, arr2], axis=1)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18616 entries, 0 to 18615
Data columns (total 53 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   player_name        18616 non-null  object 
 1   pitcher            18616 non-null  int64  
 2   game_year          18616 non-null  int64  
 3   game_date          18616 non-null  object 
 4   game_type          18616 non-null  object 
 5   home_team          18616 non-null  object 
 6   away_team          18616 non-null  object 
 7   pitch_type         18616 non-null  object 
 8   pitch_name         18616 non-null  object 
 9   pitch_number       18616 non-null  int64  
 10  release_speed      18616 non-null  float64
 11  release_spin_rate  18615 non-null  float64
 12  vx0                18616 non-null  float64
 13  vy0                18616 non-null  float64
 14  vz0                18616 non-null  float64
 15  effective_speed    18616 non-null  float64
 16  events             186

In [702]:
#Change date into a datetime object.
df2['game_date'] = pd.to_datetime(df2['game_date'])

## PART 2: A COMPARISON OF ACES

In [717]:
cole_df = df2[df2.player_name=='Gerrit Cole']
cole_df = cole_df.drop(['pitcher', 'player_name', 'game_type', 'game_year', 'home_team', 'away_team',
                        'type', 'events', 'pitch_name', 'pitch_type', 'pitch_number',
                        'effective_speed', 'description'], axis=1)
#This creates a boolean dataframe of all rows and only columns with at least 1 nonzero value.
cole_df = cole_df.loc[:, (cole_df != 0).any(axis=0)]

In [718]:
degrom_df = df2[df2.player_name=='Jacob deGrom']
degrom_df = degrom_df.drop(['pitcher', 'player_name', 'game_type', 'game_year', 'home_team', 'away_team',
                        'type', 'events', 'pitch_name', 'pitch_type', 'pitch_number',
                        'effective_speed', 'description'], axis=1)
degrom_df = degrom_df.loc[:, (degrom_df != 0).any(axis=0)]

In [722]:
degrom_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1135 entries, 9814 to 18615
Data columns (total 34 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   game_date          1135 non-null   datetime64[ns]
 1   release_speed      1135 non-null   float64       
 2   release_spin_rate  1135 non-null   float64       
 3   vx0                1135 non-null   float64       
 4   vy0                1135 non-null   float64       
 5   vz0                1135 non-null   float64       
 6   balls              1135 non-null   int64         
 7   strikes            1135 non-null   int64         
 8   zone               1135 non-null   float64       
 9   release_pos_x      1135 non-null   float64       
 10  release_pos_z      1135 non-null   float64       
 11  pfx_x              1135 non-null   float64       
 12  pfx_z              1135 non-null   float64       
 13  plate_x            1135 non-null   float64       
 14  plat

In [723]:
# Save for modeling.
cole_df.to_csv('cole_df.csv', index=False)
degrom_df.to_csv('degrom_df.csv', index=False)

In [719]:
cole_df.describe()

,release_speed,release_spin_rate,vx0,vy0,vz0,balls,strikes,zone,release_pos_x,release_pos_z,...,called_strike,double,field_error,field_out,home_run,single,strikeout,strikeout_looking,swinging_strike,walk
count,1499.000000,1499.000000,1499.000000,1499.000000,1499.000000,1499.000000,1499.000000,1499.000000,1499.000000,1499.000000,...,1499.000000,1499.000000,1499.000000,1499.000000,1499.000000,1499.000000,1499.000000,1499.000000,1499.000000,1499.000000
mean,91.955904,2537.185457,6.071839,-133.724983,-4.218497,0.861908,0.969980,8.935957,-1.893783,5.646398,...,0.145430,0.010674,0.002001,0.096731,0.012008,0.021348,0.067378,0.015344,0.286191,0.015344
std,5.545790,246.363163,2.248864,7.983508,2.811732,0.950867,0.833081,4.426363,0.134584,0.148687,...,0.352652,0.102795,0.044706,0.295690,0.108957,0.144588,0.250760,0.122956,0.452131,0.122956
min,79.800000,1494.000000,-1.409470,-145.555876,-12.791878,0.000000,0.000000,1.000000,-2.280000,5.270000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,87.600000,2477.500000,4.560260,-140.854636,-6.116760,0.000000,0.000000,5.000000,-1.990000,5.540000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,94.700000,2554.000000,6.180279,-137.662640,-4.404578,1.000000,1.000000,11.000000,-1.890000,5.630000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,96.900000,2658.500000,7.634796,-127.561971,-2.521403,1.000000,2.000000,13.000000,-1.790000,5.750000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,100.100000,2981.000000,12.345149,-116.114304,7.090214,3.000000,2.000000,14.000000,-1.530000,6.120000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [720]:
degrom_df.describe()

,release_speed,release_spin_rate,vx0,vy0,vz0,balls,strikes,zone,release_pos_x,release_pos_z,...,called_strike,double,field_error,field_out,home_run,single,strikeout,strikeout_looking,swinging_strike,walk
count,1135.000000,1135.000000,1135.000000,1135.000000,1135.000000,1135.000000,1135.000000,1135.000000,1135.000000,1135.000000,...,1135.000000,1135.000000,1135.000000,1135.000000,1135.000000,1135.000000,1135.000000,1135.000000,1135.000000,1135.000000
mean,94.816035,2370.465198,4.441610,-138.024809,-4.513351,0.875771,0.971806,9.237004,-1.108546,5.514238,...,0.119824,0.010573,0.001762,0.085463,0.006167,0.024670,0.081057,0.010573,0.331278,0.015859
std,3.807253,372.379413,1.962438,5.485615,2.276066,0.967498,0.852994,4.373768,0.126286,0.090220,...,0.324899,0.102324,0.041959,0.279692,0.078325,0.155185,0.273043,0.102324,0.470880,0.124985
min,80.700000,794.000000,-2.624003,-148.481468,-11.105840,0.000000,0.000000,1.000000,-1.600000,5.240000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,91.900000,2390.000000,3.177651,-143.287743,-6.068635,0.000000,0.000000,5.000000,-1.200000,5.450000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,93.700000,2498.000000,4.494357,-136.500216,-4.650687,1.000000,1.000000,11.000000,-1.110000,5.510000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,98.500000,2579.000000,5.684267,-133.852038,-3.220141,1.000000,2.000000,14.000000,-1.020000,5.570000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,102.200000,3556.000000,11.198119,-117.516517,5.088735,3.000000,2.000000,14.000000,-0.750000,5.830000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [721]:
cole_df.iloc[45]

game_date            2020-10-09 00:00:00
release_speed                       82.0
release_spin_rate                 2596.0
vx0                             2.696395
vy0                          -119.280085
vz0                             3.422061
balls                                  0
strikes                                0
zone                                 2.0
release_pos_x                      -1.84
release_pos_z                       5.72
pfx_x                               0.73
pfx_z                              -1.08
plate_x                            -0.03
plate_z                             3.39
release_extension                    6.5
ax                              6.364272
ay                             24.064651
az                            -43.378214
CH                                     0
FF                                     0
KC                                     1
SL                                     0
ball                                   0
called_strike   

Elite pitchers have high spin rates. Cole's is a bit higher on average but deGrom has a much larger range, suggesting he makes more use of varation, looking at the std and min.